In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time
import os
from datetime import datetime

In [2]:
# Variaveis da automação

url = "https://www.infomoney.com.br/ferramentas/altas-e-baixas/"
project_path = os.getcwd()
download_path = os.path.join(project_path, 'relatorios')

# Cria a pasta se ela não existir
os.makedirs(download_path, exist_ok=True)


# Função de renomear o arquivo para adicionar a data atual
def esperar_download_concluir(pasta, timeout=30):
    for _ in range(timeout):
        if not any(f.endswith(".crdownload") for f in os.listdir(pasta)):
            return
        time.sleep(1)
    raise TimeoutException("Download não finalizado a tempo.")

def renomear_arquivo_altas(download_path):
    nome_original = "altas_e_baixas.csv"
    caminho_original = os.path.join(download_path, nome_original)

    if os.path.exists(caminho_original):
        data = datetime.now().strftime("%d_%m_%Y")
        novo_nome = f"altas_e_baixas_{data}.csv"
        caminho_novo = os.path.join(download_path, novo_nome)

        os.rename(caminho_original, caminho_novo)
        print(f"Arquivo renomeado para: {novo_nome}")
    else:
        print(f"Arquivo {nome_original} não encontrado.")


In [ ]:
# Verificação de ADS

options = Options()

# options.add_argument("--headless=new")
options.add_argument("--load_extension=/home/felip/CavalariDev/Finance-Bot/extensions/ublock")

driver = webdriver.Chrome(service=Service(), options=options)

# Habilita o protocolo DevTools
driver.execute_cdp_cmd("Network.enable", {})

# Bloqueia requisições de anúncios
driver.execute_cdp_cmd("Network.setBlockedURLs", {"urls": ["*googleads*"]})

driver.get("https://www.infomoney.com.br/ferramentas/altas-e-baixas/")

try:
    anuncio = driver.find_element(By.ID, "OutOfPage")
    if anuncio != None:
        driver.execute_script(""" 
    const element = document.getElementById("OutOfPage")
    if (element) {
        element.style.display = 'none'}
""")
    print(anuncio)
except:
    print("não achei")


<selenium.webdriver.remote.webelement.WebElement (session="b698f934f213d41dfb7fc70386b51ca9", element="f.A6508E2BB207E19F22D56216CC5641B8.d.8F141396640E148FF75BEA4AE53459E3.e.13")>


In [ ]:
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
    "safebrowsing.enabled": True
})
# options.add_argument("--headless=new")  # 👈 novo modo headless do Chrome
options.add_argument("--window-size=1920,1080")  # Define tamanho da janela (evita bugs de layout)
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


# Iniciando automação
driver.get(url)
# driver.maximize_window()

 # Aguarda carregamento completo da tabela
time.sleep(5)

# Verificando se não existe a publicidade no site
try:
    close_banner = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "fechar")))
    close_banner.click()
except:
    print('Não encontrou o botão clicavel')

# Captando botão para download
try:
    btn = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "export_altas_e_baixas_mobile")))
    btn.click()
    print("encontrou o botão em formato mobile")
except TimeoutException:
    print("Não encontrou o formato mobile, tentando no formato Web")
    try:
        btn = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "export_altas_e_baixas")))
        btn.click()
        print("encontrou o botão em formato web")
    except:
        print('Não encontrou nem no Web')


driver.close()

## Renomeia o arquivo
renomear_arquivo_altas(download_path)


deu ruim
Não encontrou o formato mobile, tentando no formato Web
encontrou o botão em formato web
Arquivo renomeado para: altas_e_baixas_20_08_2025.csv
